In [57]:
# !pip install transformers

In [58]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc

from transformers import pipeline
import re

In [197]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()

# Load the training data
data = pd.read_csv('/kaggle/input/murderdata/Mysterious_Affair_at_Styles_Train_Data.csv')


In [198]:
data

,OrginalPage,Text
0,1,CHAPTER I.\nI GO TO STYLES\n\n\nThe intense in...
1,2,"looked his forty-five years. As a boy, though,..."
2,3,at the\ntime of their father's remarriage that...
3,4,"black beard,\nand wears patent leather boots i..."
4,5,"""I'm afraid you'll find it very quiet down her..."
...,...,...
202,203,hers! It concealed a very opposite emotion.\nT...
203,204,them to us. There was really more\nevidence ag...
204,205,something about the matter. He was nearly\ndri...
205,206,"""Yes, I quite understood your distress.""\n\n""P..."


In [196]:
# import gpt_2_simple as gpt2

# # Fine-tune the GPT-2 model on your text data
# sess = gpt2.start_tf_sess()
# gpt2.finetune(sess, dataset=data.Text, model_name='124M')

# # Generate augmented text samples
# augmented_texts = []
# for _ in range(num_samples):
#     augmented_text = gpt2.generate(sess, temperature=0.5)[0]
#     augmented_texts.append(augmented_text)

In [181]:
# import nlpaug.augmenter.word as naw
# # from textaug import SynonymAug

# # Initialize the SynonymAug augmenter
# augmenter = SynonymAug()

# # Initialize the SynonymAug augmenter
# # augmenter = naw.SynonymAug(aug_src='wordnet', aug_p=0.3)

# # Augment each text sample in the list
# augmented_texts = []
# augmented_label = []
# for i,text in enumerate(data.Text):
#     augmented_text = augmenter.augment(text)
#     augmented_texts.append(augmented_text)
#     augmented_label.append(data.loc[i,"OrginalPage"])

In [64]:
# split_data = pd.DataFrame(columns=['text', 'OrginalPage'])

In [199]:
# for _, row in data.iterrows():
#     # Split the text into two equal parts
#     text1, text2 = row['Text'][:len(row['Text']) // 2], row['Text'][len(row['Text']) // 2:]
#     # Add the split texts to the dataframe as separate rows
#     split_data = split_data.append({'text': text1, 'OrginalPage': row['OrginalPage']}, ignore_index=True)
#     split_data = split_data.append({'text': text2, 'OrginalPage': row['OrginalPage']}, ignore_index=True)
    
# split_data=split_data.sample(frac=1)
# # Print the resulting dataframe
# split_data

In [200]:
def remove_newlines(text):
    return re.sub(r'\n', '', text)

In [201]:
# Preprocess the text data by lowercasing and removing punctuation
# split_data['text'] = split_data['text'].apply(remove_newlines)
data['Text'] = data['Text'].apply(remove_newlines)

In [202]:
# X = split_data['text']
# X = X.str.lower().str.replace(r'[^\w\s]', '')
# y = split_data['OrginalPage']
X = data['Text']
X = X.str.lower().str.replace(r'[^\w\s]', '')
y = data['OrginalPage']

In [227]:
vectorizer = CountVectorizer()
tfidf_dataa = vectorizer.fit_transform(X).toarray()
features = vectorizer.get_feature_names_out()
tfidf_data = pd.DataFrame(tfidf_dataa, columns=features)

In [228]:
tfidf_data.head()

,15th,16th,16threallyi,16thwill,17th,17thbut,1904,1soverlapping,24th,29th,...,youthough,youto,youve,youwhat,youwhich,youwhile,youwill,youyes,youyou,zeal
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [229]:
# vectorizer = CountVectorizer(ngram_range=(2, 2))
# X_bigrams = vectorizer.fit_transform(X)

# Extract topic models using Latent Dirichlet Allocation
from sklearn.decomposition import LatentDirichletAllocation

# Create a LatentDirichletAllocation object
lda = LatentDirichletAllocation(n_components=207)

# Fit the model to the data and transform the text to a topic representation
X_topics = lda.fit_transform(tfidf_dataa)

In [230]:
X_topics.shape

(207, 207)

In [209]:
# X_bigrams.shape

(207, 33222)

In [231]:
tfidf_data = pd.concat([tfidf_data, pd.DataFrame(X_topics)], axis=1)

In [232]:
tfidf_data

,15th,16th,16threallyi,16thwill,17th,17thbut,1904,1soverlapping,24th,29th,...,197,198,199,200,201,202,203,204,205,206
0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
203,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
204,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
205,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [217]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest=train_test_split(tfidf_data, y, test_size=0.1, random_state=11)

In [73]:
# !pip install lazypredict

In [218]:
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import shuffle
import numpy as np

reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None )
models,predictions = reg.fit(xtrain, xtest, ytrain, ytest)

 55%|█████▍    | 23/42 [02:09<01:23,  4.38s/it]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 79%|███████▊  | 33/42 [02:35<00:16,  1.81s/it]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 186.


100%|██████████| 42/42 [02:57<00:00,  4.22s/it]


In [219]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
Lars,2496874079749963839803752448.00,-1254429537666381847191154065408.00,62644001249028160.00,1.38
PoissonRegressor,34693100390182598612090880.00,-17429813636027738504787132416.00,7384187664732426.00,0.73
GammaRegressor,12366831054638163618168832.00,-6213095921850213965267730432.00,4408698332486419.50,0.50
SGDRegressor,58825845029282789720064.00,-29554104542711672320557056.00,304064176386619.00,0.49
ElasticNetCV,17.65,-8361.95,5114.89,40.25
MLPRegressor,5.76,-2391.41,2735.73,13.45
TransformedTargetRegressor,2.47,-737.28,1519.73,0.58
LinearRegression,2.47,-737.28,1519.73,0.52
LinearSVR,2.46,-732.99,1515.31,2.11


In [263]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.linear_model import QuantileRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier , StackingRegressor, VotingRegressor
from sklearn.linear_model import OrthogonalMatchingPursuitCV, ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import GammaRegressor


model=AdaBoostRegressor()
model4=GammaRegressor()
model3 = HuberRegressor()
model2 = ElasticNetCV()
model1 = ElasticNet()
voting_model = StackingRegressor(estimators=[('model_1', model1), ('model_2', model2)
                                            , ('model_3', model3),("model_4",model)])




reg = voting_model.fit(tfidf_data, y)

In [77]:
# from sklearn.linear_model import OrthogonalMatchingPursuitCV
# reg = OrthogonalMatchingPursuitCV(cv=5).fit(tfidf_data, y)

In [234]:
# Load the test data
test_data = pd.read_csv('/kaggle/input/murderdata/Cain_s_Jawbone_Test_data.csv')
test_data['Text'] = test_data['Text'].apply(remove_newlines)
X_test = test_data['Text']
# Preprocess the test data in the same way as the training data
X_test = X_test.str.lower().str.replace(r'[^\w\s]', '')
# Extract features from the test data using the same vectorizer


In [237]:
testt = vectorizer.transform(X_test).toarray()
features = vectorizer.get_feature_names_out()
test_data = pd.DataFrame(testt, columns=features)

X_topicss = lda.transform(testt)
test_data = pd.concat([test_data, pd.DataFrame(X_topicss)], axis=1)

In [264]:
y__pred=reg.predict(test_data)

In [265]:
y__pred

array([ 87.77784274,  89.75980753, 101.44257841,  90.15683543,
        90.63311458,  88.84939727,  84.15135338,  88.73155956,
        87.30944333,  94.11549653,  91.8542575 ,  86.17420112,
        85.14806625,  94.36004848,  88.70346715,  84.4573267 ,
        88.93499078,  89.95852659,  83.97976122,  88.67115526,
        85.63855079,  94.98473164,  88.90133613,  96.95915865,
        86.61488041,  87.16251292,  94.82345909,  88.27619086,
        85.76723768,  88.64893682,  84.55214087,  84.20133947,
        97.84353708,  95.05580713,  82.87022585,  91.65108547,
        87.25443894,  91.95522394,  86.50761057,  91.79040217,
        82.53400634,  83.16933034,  86.47131649,  89.65133033,
        83.87731122,  89.39721211,  83.50762205,  91.91416329,
        83.43576483,  88.67922712,  90.82370654,  94.7593421 ,
        86.76116368,  89.83994393,  84.42717559,  91.03400171,
        87.64793924,  87.33071317,  92.13390849,  87.64257606,
        84.86308697,  89.14547917,  93.46460006,  88.66

In [268]:
pred=[round((i)) for i in y__pred]
predd=[int((i)) for i in y__pred]

In [269]:
len(set(pred)) , len(set(predd))

(16, 17)

In [256]:
len(set(pred)) , len(set(predd))

(30, 30)

In [257]:
print(set(pred) ,"\n\n\n",set(predd))

{69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 98, 99, 100, 103, 105} 


 {69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 97, 99, 103, 105}


In [243]:
len(set(pred)) , len(set(predd))

(17, 16)

In [270]:
print(set(pred) ,"\n\n\n",set(predd))

{97, 98, 101, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95} 


 {96, 97, 101, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95}


In [244]:
print(set(pred) ,"\n\n\n",set(predd))

{131, 72, 73, 74, 75, 76, 109, 78, 115, 84, 117, 120, 121, 91, 92, 93, 95} 


 {130, 131, 71, 73, 75, 77, 109, 114, 84, 117, 120, 121, 90, 92, 93, 95}


In [245]:
sub=pd.read_csv("/kaggle/input/murderdata/SampleSubmission.csv")

In [274]:
sub["CorrectPage"] = pred

In [275]:
sub.CorrectPage

0      88
1      90
2     101
3      90
4      91
     ... 
70     87
71     90
72     91
73     94
74     90
Name: CorrectPage, Length: 75, dtype: int64

In [276]:
sub.CorrectPage.nunique()

16

In [277]:
sub.to_csv("sub20.csv", index=False)

In [278]:
sub

,OriginalPage,CorrectPage
0,1,88
1,2,90
2,3,101
3,4,90
4,5,91
...,...,...
70,71,87
71,72,90
72,73,91
73,74,94
